In [ ]:
#| default_exp hpopt
#| default_cls_lvl 3

In [ ]:
#| export
from seqdata.core import *
from seqdata.models.core import *
from seqdata.learner import *
from fastai.basics import *
from fastai.callback.schedule import *
from fastai.callback.rnn import *
from fastai.callback.tracker import *

import ray
from ray import tune
from ray.tune import Trainable
from ray.tune.schedulers import *
from ray.tune.experiment.trial import ExportFormat

Library "haste_pytorch" not found


In [ ]:
f_paths = Path.cwd() / 'test_data/WienerHammerstein/'
hdf_files = L([f for f in get_hdf_files(f_paths) if '_test.hdf5' not in str(f)])
tfm_src = CreateDict([DfHDFCreateWindows(win_sz=400,stp_sz=100,clm='u')])
dls = DataBlock(blocks=(SequenceBlock.from_hdf(['u'],TensorSequencesInput),
                        SequenceBlock.from_hdf(['y'],TensorSequencesOutput)),
                get_items=tfm_src,
                splitter=ApplyToDict(FuncSplitter(lambda o: 'valid' in str(o)))).dataloaders(hdf_files)

## optimizer core

First we need a log uniform distibution for variables with vast value ranges

In [ ]:
#| export
def log_uniform(min_bound, max_bound, base=10):
    '''uniform sampling in an exponential range'''
    logmin = np.log(min_bound) / np.log(base)
    logmax = np.log(max_bound) / np.log(base)
    def _sample():
        return base**(np.random.uniform(logmin, logmax))
    return _sample

In [ ]:
[log_uniform(1e-8, 1e-2)() for _ in range(5)]

[4.360565948781439e-08,
 0.0007031243952688263,
 4.6828570547193034e-05,
 4.8789405507773396e-08,
 7.552366695077453e-05]

In [ ]:
#| export
class LearnerTrainable(tune.Trainable):

    def setup(self, config):
        self.create_lrn = ray.get(config['create_lrn'])
        self.dls = ray.get(config['dls'])

        self.lrn = self.create_lrn(self.dls,config)

    def step(self):
        with self.lrn.no_bar(): self.lrn.fit(1)
        train_loss,valid_loss,rmse = self.lrn.recorder.values[-1]
        result = {'train_loss': train_loss,
                'valid_loss': valid_loss,
                'mean_loss': rmse}
        return result

    def save_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "model.pth")
        torch.save(self.lrn.model.state_dict(), checkpoint_path)
        return tmp_checkpoint_dir

    def load_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "model.pth")
        self.lrn.model.load_state_dict(torch.load(checkpoint_path))

    def _export_model(self, export_formats, export_dir):
        if export_formats == [ExportFormat.MODEL]:
            path = os.path.join(export_dir, "exported_model")
            torch.save(self.lrn.model.state_dict(), path)
            return {ExportFormat.MODEL: path}
        else:
            raise ValueError("unexpected formats: " + str(export_formats))

    # the learner class will be recreated with every perturbation, saving the model
    # that way the new hyperparameter will be applied
    def reset_config(self, new_config):
        self.lrn = self.create_lrn(self.dls,new_config)
        self.config = new_config
        return True

In [ ]:
#| export
from fastai.callback.tracker import SaveModelCallback 
class CBRaySaveModel(SaveModelCallback):
    "A `TrackerCallback` that saves the model's best during training in a tune checkpoint directory"
    
    def _save(self, name):
        if not hasattr(self,'idx_checkpoint'): self.idx_checkpoint = 0
        with tune.checkpoint_dir(step=self.idx_checkpoint) as checkpoint_dir:
            file = os.path.join(checkpoint_dir,name+'.pth')
            print(file)
            save_model(file, self.learn.model,opt=None)
            self.last_saved_path = file
            self.idx_checkpoint += 1
            
    #final checkpoint
    def after_fit(self, **kwargs):
        self._save(f'{self.fname}')

In [ ]:
#| export
def learner_optimize(config, checkpoint_dir=None):
        create_lrn = ray.get(config['create_lrn'])
        dls = ray.get(config['dls'])
        
        #Scheduling Parameters for training the Model
        lrn_kwargs = {'n_epoch':100,'pct_start':0.5}
        for attr in ['n_epoch','pct_start']:
            if attr in config: lrn_kwargs[attr] = config[attr]

        lrn = create_lrn(dls,config)
        
        # load checkpoint data if provided
        if checkpoint_dir:
            lrn.model.load_state_dict(torch.load(tmp_checkpoint_dir))
        
        lrn.lr = config['lr'] if 'lr' in config else 3e-3
        lrn.add_cb(CBRayReporter() if 'reporter' not in config else ray.get(config['reporter'])())
        lrn.add_cb(CBRaySaveModel())
        with lrn.no_bar(): 
            ray.get(config['fit_method'])(lrn,**lrn_kwargs)

The mutation config dictionary consists of functions that sample from a distribution. In order to retrieve a dictionary with one realisation we need the function sample_config

In [ ]:
#| export
def sample_config(config):
    ret_conf = config.copy()
    for k in ret_conf:
        ret_conf[k]=ret_conf[k]()
    return ret_conf

In [ ]:
#| export
class CBRayReporter(Callback):
    "`Callback` reports progress after every epoch to the ray tune logger"
    
    order=70 #order has to be >50, to be executed after the recorder callback

    def after_epoch(self):
        train_loss,valid_loss,rmse = self.learn.recorder.values[-1]
        tune.report(train_loss=train_loss,
                        valid_loss=valid_loss,
                        mean_loss=rmse)

In [ ]:
#| export
class HPOptimizer():
    def __init__(self,create_lrn,dls):
        self.create_lrn = create_lrn
        self.dls = dls
        self.analysis = None
    
    @delegates(ray.init)
    def start_ray(self,**kwargs):
        ray.shutdown()
        ray.init(**kwargs)
        
    def stop_ray(self):
        ray.shutdown()
        
    
        

    @delegates(tune.run, keep=True)
    def optimize(self,config,optimize_func=learner_optimize,resources_per_trial={"gpu": 1.0},verbose=1,**kwargs):
        config['create_lrn'] = ray.put(self.create_lrn)
        #dls are large objects, letting ray handle the copying process makes it much faster
        config['dls'] = ray.put(self.dls) 
        if 'fit_method' not in config: config['fit_method'] = ray.put(Learner.fit_flat_cos)

        self.analysis = tune.run(
            optimize_func,
            config=config,
            resources_per_trial=resources_per_trial,
            verbose=verbose,
            **kwargs)
        return self.analysis
        
    @delegates(tune.run, keep=True)
    def optimize_pbt(self,opt_name,num_samples,config,mut_conf,perturbation_interval=2,
                 stop={"training_iteration": 40 },
                 resources_per_trial={"gpu": 1 },
                 resample_probability=0.25,
                 quantile_fraction=0.25,
                 **kwargs):
        self.mut_conf = mut_conf
        
        config['create_lrn'] = ray.put(self.create_lrn)
        #dls are large objects, letting ray handle the copying process makes it much faster
        config['dls'] = ray.put(self.dls) 
        
        

        
        scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="mean_loss",
        mode="min",
        perturbation_interval=perturbation_interval,
        resample_probability=resample_probability,
        quantile_fraction=quantile_fraction,
        hyperparam_mutations=mut_conf)
        
        self.analysis = tune.run(
            LearnerTrainable,
            name=opt_name,
            scheduler=scheduler,
            reuse_actors=True,
            verbose=1,
            stop=stop,
            checkpoint_score_attr="mean_loss",
            num_samples=num_samples,
            resources_per_trial=resources_per_trial,
            config=config,
            **kwargs)
        return self.analysis
    
    def best_model(self):
        if self.analysis is None: raise Exception
        model = self.create_lrn(self.dls,sample_config(self.mut_conf)).model
        f_path = ray.get(self.analysis.get_best_trial('mean_loss',mode='min').checkpoint.value)
        model.load_state_dict(torch.load(f_path))
        return model

### Test Population Based Training

In [ ]:
def create_lrn(dls,config):
    lr = config['lr']
    alpha = config['alpha']
    beta = config['beta']
    weight_p = config['weight_p']
    
    lrn = RNNLearner(dls)
    lrn.lr = lr
    return lrn

In [ ]:
config={
            "lr": tune.loguniform(1e-2, 1e-4),
            "alpha": tune.loguniform(1e-5, 10),
            "beta": tune.loguniform(1e-5, 10),
            "weight_p": tune.uniform(0, 0.5)}
mut_conf = {# distribution for resampling
            "lr": log_uniform(1e-8, 1e-2),
            "alpha": log_uniform(1e-5, 10),
            "beta": log_uniform(1e-5, 10),
            "weight_p": lambda: np.random.uniform(0, 0.5)}

hp_opt = HPOptimizer(create_lrn,dls)
hp_opt.start_ray()
hp_opt.optimize_pbt('pbt_test',4,config,mut_conf,perturbation_interval=1,
                 stop={"training_iteration": 3 },
                 resources_per_trial={"gpu": 0.5},
                 local_dir=Path.home() / 'ray_results')#no cpu count is necessary

(LearnerTrainable pid=2016777) Library "haste_pytorch" not found
(LearnerTrainable pid=2016777) [0, 0.05623083561658859, 0.06141030788421631, 0.24773479998111725, '00:01']
(LearnerTrainable pid=2016777) [0, 0.055782824754714966, 0.06067557632923126, 0.24623264372348785, '00:00']
(LearnerTrainable pid=2016777) [0, 0.055266521871089935, 0.06033457815647125, 0.24552637338638306, '00:00']
(LearnerTrainable pid=2017021) Library "haste_pytorch" not found [repeated 3x across cluster]


2023-06-02 11:42:14,297	INFO pbt.py:808 -- 

[PopulationBasedTraining] [Exploit] Cloning trial baea2_00003 (score = -0.246968) into trial baea2_00001 (score = -0.248813)

2023-06-02 11:42:14,298	INFO pbt.py:835 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of trialbaea2_00001:
lr : 0.0003583798133123241 --- (resample) --> 0.0008133846763418052
alpha : 2.630345446261178 --- (resample) --> 0.0004267946720993688
beta : 0.4277416920291437 --- (resample) --> 0.00040512506873475264
weight_p : 0.3292983861228925 --- (* 0.8) --> 0.263438708898314

2023-06-02 11:42:14,300	WARNING trial_runner.py:1543 -- You are trying to access pause_trial interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A more strict API access pattern would be enforced starting 1.12s.0


(LearnerTrainable pid=2017023) [0, 0.05556301400065422, 0.06058104336261749, 0.24602438509464264, '00:00'] [repeated 5x across cluster]


(LearnerTrainable pid=2017023) 2023-06-02 11:42:14,885	INFO trainable.py:913 -- Restored on 141.23.125.123 from checkpoint: /tmp/checkpoint_tmp_8c56e66bcc4544ca9bcf579ea7d0cd33
(LearnerTrainable pid=2017023) 2023-06-02 11:42:14,885	INFO trainable.py:922 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': None, '_time_total': 1.769798994064331, '_episodes_total': None}


(LearnerTrainable pid=2054475) [0, 0.05511416494846344, 0.06048772484064102, 0.24583807587623596, '00:01']
(LearnerTrainable pid=2054475) [0, 0.05511416494846344, 0.06048772484064102, 0.24583807587623596, '00:01'] [repeated 5x across cluster]


2023-06-02 11:42:20,781	INFO tune.py:945 -- Total run time: 17.16 seconds (17.14 seconds for the tuning loop).


<ray.tune.analysis.experiment_analysis.ExperimentAnalysis>

In [ ]:
#hp_opt.best_model()

### Test Grid Search

In [ ]:
def create_lrn(dls,config):
    lrn = RNNLearner(dls,hidden_size=config['hidden_size'],metrics=[fun_rmse,mse])
    return lrn

In [ ]:
class CustomReporter(Callback):
    "`Callback` reports progress after every epoch to the ray tune logger"
    
    order=70 #order has to be >50, to be executed after the recorder callback

    def after_epoch(self):
        train_loss,valid_loss,rmse,mse = self.learn.recorder.values[-1]
        tune.report(train_loss=train_loss,
                        valid_loss=valid_loss,
                        mean_loss=rmse,
                        mse=mse)

In [ ]:
hp_opt = HPOptimizer(create_lrn,dls)

In [ ]:
hp_opt.start_ray(local_mode=False)

(LearnerTrainable pid=2054475) 2023-06-02 11:42:19,031	INFO trainable.py:913 -- Restored on 141.23.125.123 from checkpoint: /tmp/checkpoint_tmp_e10d019bdb3c4142a570321db81ebfe8
(LearnerTrainable pid=2054475) 2023-06-02 11:42:19,031	INFO trainable.py:922 -- Current state after restoring: {'_iteration': 2, '_timesteps_total': None, '_time_total': 2.717446804046631, '_episodes_total': None}
2023-06-02 11:42:30,256	INFO worker.py:1625 -- Started a local Ray instance.


In [ ]:
search_space = {
    "hidden_size": tune.grid_search([10,20,50,100]),
    'n_epoch':10,
    'reporter':ray.put(CustomReporter)
}

In [ ]:
hp_opt.optimize(resources_per_trial={"gpu": 0.5},
                config=search_space)

(learner_optimize pid=2080632) Library "haste_pytorch" not found
(learner_optimize pid=2080632) [0, 0.1300196796655655, 0.13209111988544464, 0.36324384808540344, 0.13209111988544464, '00:01']
(learner_optimize pid=2080632) Better model found at epoch 0 with valid_loss value: 0.13209111988544464.
(learner_optimize pid=2080632) /home/pheenix/ray_results/learner_optimize_2023-06-02_11-42-39/learner_optimize_d06c4_00000_0_hidden_size=10_2023-06-02_11-42-39/checkpoint_000000/model.pth
(learner_optimize pid=2080632) [1, 0.1227642297744751, 0.1166750118136406, 0.34136971831321716, 0.1166750118136406, '00:00']
(learner_optimize pid=2080632) Better model found at epoch 1 with valid_loss value: 0.1166750118136406.
(learner_optimize pid=2080632) /home/pheenix/ray_results/learner_optimize_2023-06-02_11-42-39/learner_optimize_d06c4_00000_0_hidden_size=10_2023-06-02_11-42-39/checkpoint_000001/model.pth
(learner_optimize pid=2080632) [2, 0.11328967660665512, 0.09937257319688797, 0.31502074003219604, 

2023-06-02 11:42:58,873	INFO tune.py:945 -- Total run time: 19.18 seconds (19.14 seconds for the tuning loop).


<ray.tune.analysis.experiment_analysis.ExperimentAnalysis>

In [ ]:
hp_opt.analysis.get_best_config('mean_loss',mode='min')

{'hidden_size': 100,
 'n_epoch': 10,
 'reporter': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000005000000),
 'create_lrn': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000006000000),
 'dls': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000007000000),
 'fit_method': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000008000000)}

### Test Random Search

In [ ]:
#| include: false
import nbdev; nbdev.nbdev_export()